## Data Science for Good - Text classification using PySpark ML

In [8]:
# ipmortant libraries
!#pip install pyspark


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.

import os
for dirname, _, filenames in os.walk("/media/mnegm/Negmo's Files/Artificial intelligence/Negmo's workspace/Python/Pysparkpro/data"):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/media/mnegm/Negmo's Files/Artificial intelligence/Negmo's workspace/Python/Pysparkpro/data/sample_submission.csv
/media/mnegm/Negmo's Files/Artificial intelligence/Negmo's workspace/Python/Pysparkpro/data/test.csv
/media/mnegm/Negmo's Files/Artificial intelligence/Negmo's workspace/Python/Pysparkpro/data/train.csv


In [2]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import CountVectorizer,StringIndexer, RegexTokenizer,StopWordsRemover
from pyspark.sql.functions import col, udf,regexp_replace,isnull
from pyspark.sql.types import StringType,IntegerType
from pyspark.ml.classification import NaiveBayes, RandomForestClassifier, LogisticRegression, DecisionTreeClassifier, GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

In [3]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

22/12/20 23:17:02 WARN Utils: Your hostname, mnegm-Latitude-E7440 resolves to a loopback address: 127.0.1.1, but we couldn't find any external IP address!
22/12/20 23:17:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/20 23:17:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/20 23:17:05 WARN MacAddressUtil: Failed to find a usable hardware address from the network interfaces; using random bytes: 31:30:2d:97:61:50:63:bc


In [4]:
filepath = "/media/mnegm/Negmo's Files/Artificial intelligence/Negmo's workspace/Python/Pysparkpro/data"
sdf_train = spark.read.csv(f'{filepath}/train.csv', header = True, inferSchema = True)
sdf_test = spark.read.csv(f'{filepath}/test.csv', inferSchema=True, header=True)

sdf_sample_submission = spark.read.csv(f'{filepath}/sample_submission.csv', inferSchema=True, header=True)
sdf_train.printSchema()

root
 |-- id: string (nullable = true)
 |-- keyword: string (nullable = true)
 |-- location: string (nullable = true)
 |-- text: string (nullable = true)
 |-- target: integer (nullable = true)



In [5]:
import pandas as pd
pd.DataFrame(sdf_train.take(5), columns=sdf_train.columns)

,id,keyword,location,text,target
0,1,None,None,Our Deeds are the Reason of this #earthquake M...,1
1,4,None,None,Forest fire near La Ronge Sask. Canada,1
2,5,None,None,All residents asked to 'shelter in place' are ...,1
3,6,None,None,"13,000 people receive #wildfires evacuation or...",1
4,7,None,None,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
print("Training Data Record Count:",sdf_train.count())
print("Test Data Record Count:",sdf_test.count())


Training Data Record Count: 8387
Test Data Record Count: 3613


In [7]:
sdf_train.toPandas().groupby(['target']).size()

target
0.0    4095
1.0    3081
dtype: int64

In [8]:
ml_df = sdf_train.select("id","text","target")
ml_df.show(5)

+---+--------------------+------+
| id|                text|target|
+---+--------------------+------+
|  1|Our Deeds are the...|     1|
|  4|Forest fire near ...|     1|
|  5|All residents ask...|     1|
|  6|13,000 people rec...|     1|
|  7|Just got sent thi...|     1|
+---+--------------------+------+
only showing top 5 rows



In [9]:
ml_df = ml_df.dropna()
ml_df.count()

7176

In [10]:
ml_df = ml_df.withColumn("only_str",regexp_replace(col('text'), '\d+', ''))
ml_df.show(5)

+---+--------------------+------+--------------------+
| id|                text|target|            only_str|
+---+--------------------+------+--------------------+
|  1|Our Deeds are the...|     1|Our Deeds are the...|
|  4|Forest fire near ...|     1|Forest fire near ...|
|  5|All residents ask...|     1|All residents ask...|
|  6|13,000 people rec...|     1|, people receive ...|
|  7|Just got sent thi...|     1|Just got sent thi...|
+---+--------------------+------+--------------------+
only showing top 5 rows



In [11]:
regex_tokenizer = RegexTokenizer(inputCol="only_str", outputCol="words", pattern="\\W")
raw_words = regex_tokenizer.transform(ml_df)
raw_words.show(5)

+---+--------------------+------+--------------------+--------------------+
| id|                text|target|            only_str|               words|
+---+--------------------+------+--------------------+--------------------+
|  1|Our Deeds are the...|     1|Our Deeds are the...|[our, deeds, are,...|
|  4|Forest fire near ...|     1|Forest fire near ...|[forest, fire, ne...|
|  5|All residents ask...|     1|All residents ask...|[all, residents, ...|
|  6|13,000 people rec...|     1|, people receive ...|[people, receive,...|
|  7|Just got sent thi...|     1|Just got sent thi...|[just, got, sent,...|
+---+--------------------+------+--------------------+--------------------+
only showing top 5 rows



In [12]:
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
words_df = remover.transform(raw_words)
words_df.select("id","words","target","filtered").show(5, truncate=False)

+---+---------------------------------------------------------------------------------------------------------------------------------------------------------+------+----------------------------------------------------------------------------------------------------+
|id |words                                                                                                                                                    |target|filtered                                                                                            |
+---+---------------------------------------------------------------------------------------------------------------------------------------------------------+------+----------------------------------------------------------------------------------------------------+
|1  |[our, deeds, are, the, reason, of, this, earthquake, may, allah, forgive, us, all]                                                                       |1     |[deeds, reason, earthquake, ma

In [13]:
cv = CountVectorizer(inputCol="filtered", outputCol="features")
model = cv.fit(words_df)
countVectorizer_train = model.transform(words_df)
countVectorizer_train = countVectorizer_train.withColumn("label",col('target'))
countVectorizer_train.show(5)

+---+--------------------+------+--------------------+--------------------+--------------------+--------------------+-----+
| id|                text|target|            only_str|               words|            filtered|            features|label|
+---+--------------------+------+--------------------+--------------------+--------------------+--------------------+-----+
|  1|Our Deeds are the...|     1|Our Deeds are the...|[our, deeds, are,...|[deeds, reason, e...|(19663,[24,65,148...|    1|
|  4|Forest fire near ...|     1|Forest fire near ...|[forest, fire, ne...|[forest, fire, ne...|(19663,[7,100,140...|    1|
|  5|All residents ask...|     1|All residents ask...|[all, residents, ...|[residents, asked...|(19663,[156,578,9...|    1|
|  6|13,000 people rec...|     1|, people receive ...|[people, receive,...|[people, receive,...|(19663,[11,23,156...|    1|
|  7|Just got sent thi...|     1|Just got sent thi...|[just, got, sent,...|[got, sent, photo...|(19663,[32,102,15...|    1|
+---+---

In [14]:
countVectorizer_train.select('text','words','filtered','features','target').show()

+--------------------+--------------------+--------------------+--------------------+------+
|                text|               words|            filtered|            features|target|
+--------------------+--------------------+--------------------+--------------------+------+
|Our Deeds are the...|[our, deeds, are,...|[deeds, reason, e...|(19663,[24,65,148...|     1|
|Forest fire near ...|[forest, fire, ne...|[forest, fire, ne...|(19663,[7,100,140...|     1|
|All residents ask...|[all, residents, ...|[residents, asked...|(19663,[156,578,9...|     1|
|13,000 people rec...|[people, receive,...|[people, receive,...|(19663,[11,23,156...|     1|
|Just got sent thi...|[just, got, sent,...|[got, sent, photo...|(19663,[32,102,15...|     1|
|#RockyFire Update...|[rockyfire, updat...|[rockyfire, updat...|(19663,[7,23,284,...|     1|
|#flood #disaster ...|[flood, disaster,...|[flood, disaster,...|(19663,[17,121,17...|     1|
|I'm on top of the...|[i, m, on, top, o...|[m, top, hill, se...|(19663

In [15]:
(train, validate) = countVectorizer_train.randomSplit([0.8, 0.2],seed = 97435)

In [16]:
trainData = countVectorizer_train

#cleaning and preparing the test data
testData = sdf_test.select("id","text").dropna()
testData = testData.withColumn("only_str",regexp_replace(col('text'), '\d+', ''))
regex_tokenizer = RegexTokenizer(inputCol="only_str", outputCol="words", pattern="\\W")  #Extracting raw words
testData = regex_tokenizer.transform(testData)
remover = StopWordsRemover(inputCol="words", outputCol="filtered") #Removing stop words
testData = remover.transform(testData)
cv = CountVectorizer(inputCol="filtered", outputCol="features")
model = cv.fit(testData)
countVectorizer_test = model.transform(testData)
testData = countVectorizer_test
testData.show(5)

+---+--------------------+--------------------+--------------------+--------------------+--------------------+
| id|                text|            only_str|               words|            filtered|            features|
+---+--------------------+--------------------+--------------------+--------------------+--------------------+
|  0|Just happened a t...|Just happened a t...|[just, happened, ...|[happened, terrib...|(11358,[32,55,469...|
|  2|Heard about #eart...|Heard about #eart...|[heard, about, ea...|[heard, earthquak...|(11358,[181,196,4...|
|  3|there is a forest...|there is a forest...|[there, is, a, fo...|[forest, fire, sp...|(11358,[7,54,338,...|
|  9|Apocalypse lighti...|Apocalypse lighti...|[apocalypse, ligh...|[apocalypse, ligh...|(11358,[179,1039,...|
| 11|Typhoon Soudelor ...|Typhoon Soudelor ...|[typhoon, soudelo...|[typhoon, soudelo...|(11358,[153,201,2...|
+---+--------------------+--------------------+--------------------+--------------------+--------------------+
o

In [17]:
nb = NaiveBayes(modelType="multinomial",labelCol="label", featuresCol="features")
nbModel = nb.fit(train)
nb_predictions = nbModel.transform(validate)

In [18]:
nbEval = BinaryClassificationEvaluator()
print('Test Area Under ROC', nbEval.evaluate(nb_predictions))

Test Area Under ROC 0.4302179682948281


In [19]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
nb_accuracy = evaluator.evaluate(nb_predictions)
print("Accuracy of NaiveBayes is = %g"% (nb_accuracy))

Accuracy of NaiveBayes is = 0.803413
